### Обучение pix2pix с нуля на датасете facades (из статьи)



In [ ]:
!rm -rf /content/sample_data
!rm -rf .config
!git clone https://github.com/jys1670/pix2pix_DLS /content

In [ ]:
!python train.py

Использование генератора на произвольных изображениях:

In [ ]:
!mkdir -p output/facades/results/mask_only

In [ ]:
!python generate.py configs/default_config.py dataset/facades/mask_only output/facades/results/mask_only

### Обучение pix2pix с нуля на самодельном аниме датасете

In [ ]:
!python train.py -c configs/anime_monster.py

Использование генератора на соотв. манге:

In [ ]:
!mkdir -p output/anime_monster/results/manga/original
!python generate.py configs/anime_monster.py dataset/anime_monster/manga/original output/anime_monster/results/manga/original

In [ ]:
!mkdir -p output/anime_monster/results/manga/sketch
!python generate.py configs/anime_monster.py dataset/anime_monster/manga/sketch output/anime_monster/results/manga/sketch

### Сохранить весь output

In [ ]:
!tar czf output.tar.gz output

### Восстановить весь output

In [ ]:
!gdown -O /content/output.tar.gz https://drive.google.com/uc?id=1-5ZFOdCRgfoZZpmh2Dm73nA0k4eV5MK7

In [ ]:
!tar -xvzf output.tar.gz

Удалить все, кроме весов:

In [ ]:
!rm -rf /content/output/anime_monster/results
!rm -rf /content/output/facades/results

### Как делался аниме датасет

In [ ]:
# Серии
!cp /content/drive/MyDrive/videos.zip .
# Манга
!cp /content/drive/MyDrive/manga_parts.zip .

In [ ]:
!mkdir -p /content/dataset/anime_monster/original
!mkdir /content/dataset/anime_monster/sketch
!mkdir -p /content/dataset/anime_monster/manga/original
!mkdir -p /content/dataset/anime_monster/manga/sketch
!mkdir /content/videos

In [ ]:
!unzip manga_parts.zip -d /content/dataset/anime_monster/manga/original
!unzip videos.zip -d /content/videos

Извлечем кадры из серий:

In [ ]:
import os
import subprocess

for root, dirs, files in os.walk("/content/videos"):
    for f in files:
        fpath = str(os.path.join(root, f))
        fname = f.split(".")[0]
        command = (
            "ffmpeg -i "
            + fpath
            + ' -ss 00:06:00.000 -to 00:21:00.000 -r 1 -filter:v "crop=480:480, scale=512:512" "/content/dataset/anime_monster/original/'
            + fname
            + '_%d.bmp"'
        )
        print(" ++++++++++++ FILE", f)
        p = subprocess.Popen(
            command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
        )
        print(p.communicate()[0].decode("ascii"), end="")


Теперь надо превратить кадры в зарисовки, чтоб они хоть как-то напоминали стиль манги. Для этих целей используем Anime2Sketch. Скачаем предобученную модель:

In [ ]:
!git clone https://github.com/Mukosame/Anime2Sketch
!gdown -O /content/Anime2Sketch/weights/netG.pth https://drive.google.com/uc?id=1RILKwUdjjBBngB17JHwhZNBEaW4Mr-Ml

In [ ]:
%cd /content/Anime2Sketch
!pip install -r requirements.txt

Запустим процесс превращения исходных картинок в зарисовки:

In [ ]:
# В data.py есть ошибка
!python3 test.py --dataroot /content/dataset/anime_monster/original --load_size 512 --output_dir /content/dataset/anime_monster/sketch -g 0

Аналогично для манги (на всякий случай):

In [ ]:
!python3 test.py --dataroot /content/dataset/anime_monster/manga/original --load_size 512 --output_dir /content/dataset/anime_monster/manga/sketch -g 0

In [ ]:
%cd /content

In [ ]:
!tar czf anime_monster_dataset.tar.gz ./dataset/anime_monster